## Preliminaries

### The most important Jupyter notebook tricks
 - Press Esc to escape edit mode and add / remove cells etc.
 - When not in Edit mode: A inserts a cell above, B below
 - When not in Edit mode: DD deletes a cell
 - When in Edit mode: Shift + Enter executes the cell
 - Interrupt a hung kernel with the Stop button
 - Or restart under Kernel -> Restart

### The most important bash tricks

In [160]:
# pipe output to the next command with |
# provides the seed as input to the "mnemonic-new command"
bx seed | bx mnemonic-new

# capture output in variables with var=$(...). Notice the missing spaces around =
var=$(bx seed)

# later refer to the variable with $. echo prints stuff
echo $var

gadget deputy suspect page keep gloom wealth noble width organ tragic concert wire dismiss order raccoon this erosion
9c138f60635b1d50ceb35699ccf39b257ed7c840ec2dba1c


### Explore the bx tool

In [158]:
bx help


Usage: bx COMMAND [--help]

Version: 3.2.0

Info: The bx commands are:

address-decode
address-embed
address-encode
base16-decode
base16-encode
base58-decode
base58-encode
base58check-decode
base58check-encode
base64-decode
base64-encode
bitcoin160
bitcoin256
btc-to-satoshi
cert-new
cert-public
ec-add
ec-add-secrets
ec-multiply
ec-multiply-secrets
ec-new
ec-to-address
ec-to-ek
ec-to-public
ec-to-wif
ek-address
ek-new
ek-public
ek-public-to-address
ek-public-to-ec
ek-to-address
ek-to-ec
fetch-balance
fetch-header
fetch-height
fetch-history
fetch-public-key
fetch-stealth
fetch-tx
fetch-tx-index
fetch-utxo
hd-new
hd-private
hd-public
hd-to-ec
hd-to-public
help
input-set
input-sign
input-validate
message-sign
message-validate
mnemonic-new
mnemonic-to-seed
qrcode
ripemd160
satoshi-to-btc
script-decode
script-encode
script-to-address
seed
send-tx
send-tx-node
send-tx-p2p
settings
sha160
sha256
sha512
stealth-decode
stealth-encode
stealth-public
stealth-secret
stealth-shared
token-new
tx-dec

In [159]:
bx help tx-sign


Usage: bx tx-sign [-h] [--config value] EC_PRIVATE_KEY [TRANSACTION]...  

Info: Sign a set of transactions using a private key. Output is suitable 
for sending to Bitcoin network.                                          

Options (named):

-c [--config]        The path to the configuration settings file.        
-h [--help]          Get a description and instructions for this command.

Arguments (positional):

EC_PRIVATE_KEY       The EC private key to be used for signing.          
TRANSACTION          The set of Base16 transactions. If not specified the
                     transactions are read from STDIN.                   


### Ensure testnet is configured

In [83]:
export BX_CONFIG="../bx_config_files/bx_testnet.cfg"
bx fetch-height

2104368


# Transaction Build - BX Example
In this BX tutorial, we construct a transaction which spends one P2PKH output to three (not six) other outputs.

<img src="images/first_transaction.jpg" alt="drawing" style="" width="700px"/>



##  1. Locate your UTXO
A testnet address has been funded for you. Use the `sha256` value of your email address as a private key and derive the corresponding public key, as well as testnet P2PKH address. Find the UTXO that is locked to that P2PKH output script.

In [121]:
# Find your address
email="craig@wright.fraud"
privkey=$(echo -n $email | sha256sum | awk '{print $1}')
pubkey=$(echo $privkey | bx ec-to-public)
pubkeyhash=$(echo $pubkey | bx sha256 | bx ripemd160)
address=$(echo $pubkey | bx ec-to-address)
echo $pubkeyhash
echo $address

348d7027143ed17c8e7f211e1455d0ab5df0618f
mkJpsKY91Bz57Kg9RPod6U6vUDLhwS2JqG


In [126]:
echo $address | bx fetch-history --format json

{
    "transfers": [
        {
            "received": {
                "hash": "af22c1f2594025f9be9180b5e94e52f991a35e3336b9764f5db10cae76e2b0b4",
                "height": "2104364",
                "index": "1"
            },
            "value": "100000"
        }
    ]
}


In [139]:
# Save variables for later when we construct the transaction
previous_txid=$(echo $address | bx fetch-history --format json | jq -r ".transfers[0].received.hash")
echo $previous_txid
previous_output_index=$(echo $address | bx fetch-history --format json | jq -r ".transfers[0].received.index")
echo $previous_output_index
previous_output_amount=$(echo $address | bx fetch-history --format json | jq -r ".transfers[0].value")
echo $previous_output_amount

af22c1f2594025f9be9180b5e94e52f991a35e3336b9764f5db10cae76e2b0b4
1
100000


<hr style="border: 0.5px dashed #000;">

## 1. Setup receiving wallet.


### Create new mnemonic and derive hd master keys.

In [140]:
my_mnemonic=$(bx seed | bx mnemonic-new)
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)
echo $my_mnemonic
echo $hd_master_private
echo $hd_master_public

fox top picnic alone settle equal roast album knife april style course modify flame wide illness rocket wink
tprv8ZgxMBicQKsPeDiHqrTreAtDMMmdfAR4nQBdtDBDrVuXZUxscLFykMbWX97RLakhwFN2CwK5W3vrMELgAsGusPy7dKy1hD43VFHEwkT3Lh9
tpubD6NzVbkrYhZ4Xgk5jW8T3aYKvPHZpVbyMhnRAjDXGmhvPyDeEj5ZvrDNhJknwPBTTfiRkp1QpRvbQiF6Voqau89cNftaRtemZeQJUbxGS9Y


### Account 1: Generate destination key pairs.

You will spend the previously received utxo to receiving addresses from `account 1`, namely `/44'/1'/1'/0/0`, `/44'/1'/1'/0/1`, `/44'/1'/1'/0/2`, `/44'/1'/1'/0/3` ...


In [141]:
# Account 1 receiving parent: m/44'/1'/1'/0
hd_m_44h_1h_1h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0)

hd_m_44h_1h_1h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_2=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx sha256 | bx ripemd160)

## 2. Spending a P2PKH output.

### Create output scripts.

* **Compose P2PKH(destination public key hash) output scripts for each destination.**

In [142]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"
output_script0=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_0"] EQUALVERIFY CHECKSIG")
output_script1=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_1"] EQUALVERIFY CHECKSIG")
output_script2=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_2"] EQUALVERIFY CHECKSIG")

echo $output_script2


76a914ee1b31eff27de6192871206ea60500b85d932c6f88ac


In [143]:
# spent amount evenly across outputs, minus some sats to cover the fee
single_output_amount=$(expr $previous_output_amount / 3 - 1000)
echo $single_output_amount


32333


### Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [144]:
my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script0:$single_output_amount \
--output $output_script1:$single_output_amount \
--output $output_script2:$single_output_amount)
echo $my_tx

0100000001b4b0e276ae0cb15d4f76b936335ea391f9524ee9b58091bef9254059f2c122af0100000000ffffffff034d7e0000000000001976a914ead17789d82625acb037d22fe64e1f752241ff9888ac4d7e0000000000001976a914541968c7c7db59363c9ab4904759de593015123988ac4d7e0000000000001976a914ee1b31eff27de6192871206ea60500b85d932c6f88ac00000000


### Sign transaction.

In [145]:
bx fetch-tx --format json $previous_txid

{
    "transaction": {
        "hash": "af22c1f2594025f9be9180b5e94e52f991a35e3336b9764f5db10cae76e2b0b4",
        "inputs": [
            {
                "previous_output": {
                    "hash": "970edd89e7ec8bbce6b546d0dbca53311e724e16987b7404035002ab34aba6f2",
                    "index": "1"
                },
                "script": "[0014ce807e904fb0a38e159457182655c59f965fa2a9]",
                "sequence": "4294967294"
            }
        ],
        "lock_time": "2104363",
        "outputs": [
            {
                "address_hash": "43896e4a2ac4ca2aec605f2019c6cbf1741311c0",
                "script": "hash160 [43896e4a2ac4ca2aec605f2019c6cbf1741311c0] equal",
                "value": "3077112"
            },
            {
                "address_hash": "348d7027143ed17c8e7f211e1455d0ab5df0618f",
                "script": "dup hash160 [348d7027143ed17c8e7f211e1455d0ab5df0618f] equalverify checksig",
                "value": "100000"
            }
        ],

**Fetch previous output script (check output index ! )**

In [147]:
previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[1].script")

echo $previous_output_script
echo $pubkeyhash

dup hash160 [348d7027143ed17c8e7f211e1455d0ab5df0618f] equalverify checksig
348d7027143ed17c8e7f211e1455d0ab5df0618f


**Sign transaction with previous output script.**

In [148]:
# bx input-sign [private key] "previous output script" [transaction template]
signature=$(bx input-sign --sign_type all --index 0 $privkey "$previous_output_script" $my_tx)


In [149]:
echo $signature

304402200aa158371fa0cdbae5075ce3fd7108ba057cedb3dfaa50b5f30a27654b68757202202ba43b2c0dd59dec6cb2a1bc2b1512d3e1dc7bdaab6f0c4b650f924de27925a901


**Set the input script into the finalised transaction.**

In [150]:
# bx input-set "[signature] [public key point]" [transaction template]
my_tx=$(bx input-set --index 0 "[$signature] [$pubkey]" $my_tx)
echo $my_tx

0100000001b4b0e276ae0cb15d4f76b936335ea391f9524ee9b58091bef9254059f2c122af010000006a47304402200aa158371fa0cdbae5075ce3fd7108ba057cedb3dfaa50b5f30a27654b68757202202ba43b2c0dd59dec6cb2a1bc2b1512d3e1dc7bdaab6f0c4b650f924de27925a90121035f00bb1d48abc790b57c4f202e336b3447613fed7828328bea8b9ad5714c6a00ffffffff034d7e0000000000001976a914ead17789d82625acb037d22fe64e1f752241ff9888ac4d7e0000000000001976a914541968c7c7db59363c9ab4904759de593015123988ac4d7e0000000000001976a914ee1b31eff27de6192871206ea60500b85d932c6f88ac00000000


## 3. Validate & Broadcast the endorsed transaction

In [151]:
bx validate-tx $my_tx

The transaction is valid.


In [152]:
bx send-tx $my_tx

Sent transaction.


## Check history for sending and a receiving address

In [153]:
first_receveing_address=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx ec-to-address)
echo $first_receveing_address

n2vZLxgxpZ8RqzgUiHE1QrkZy1kwFPAwL7


In [154]:
echo $address | bx fetch-history
echo $first_receveing_address | bx fetch-history

transfers
{
    transfer
    {
        received
        {
            hash af22c1f2594025f9be9180b5e94e52f991a35e3336b9764f5db10cae76e2b0b4
            height 2104364
            index 1
        }
        value 100000
    }
}
transfers ""
